**Installs**

In [1]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.9 MB/s eta 0:00:00


In [2]:
%load_ext tensorboard

**Imports**

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from kerastuner.tuners import Hyperband
from kerastuner.tuners import BayesianOptimization

<ipython-input-3-76f30b46ddba>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


**Load dataset**

In [4]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


**Pre-process data**

In [5]:
# Preprocess the data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

**Set Constants**

In [6]:
# Constants
input_shape = (28, 28, 1)
batch_size = 128
epochs = 5
cnn_log_dir = "./logs/cnn"
fnn_log_dir = "./logs/fnn"

In [7]:
# TensorBoard callback
tensorboard_cnn_callback = TensorBoard(log_dir=cnn_log_dir, histogram_freq=1)
tensorboard_fnn_callback = TensorBoard(log_dir=fnn_log_dir, histogram_freq=1)

**Define Model Builders**

In [8]:
# Define model builder functions
def build_simple_cnn(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                                     kernel_size=hp.Choice('conv1_kernel', values=[3, 5]),
                                     activation='relu',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=hp.Int('dense_units', min_value=64, max_value=256, step=32), activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def build_feedforward_nn(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=input_shape))
    model.add(tf.keras.layers.Dense(units=hp.Int('dense_units', min_value=256, max_value=1024, step=128), activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

**Using Bayesian Optimizer Tuner to determine Optimal Hyperpameters**

**CNN**

In [9]:
bayesian_cnn_tuner = BayesianOptimization(
    build_simple_cnn,
    objective='val_accuracy',
    max_trials=5,
    directory='bayesian_optimization_logs',
    project_name='bayesian_optimization'
)

In [ ]:
bayesian_cnn_tuner.search(x=x_train, y=y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test),
                        callbacks=[tensorboard_cnn_callback])

Trial 3 Complete [00h 06m 10s]
val_accuracy: 0.8992999792098999

Best val_accuracy So Far: 0.90829998254776
Total elapsed time: 00h 17m 48s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
112               |128               |conv1_filters
3                 |3                 |conv1_kernel
224               |64                |dense_units
0.001             |0.001             |learning_rate

Epoch 1/5
469/469 [==============================] - 138s 292ms/step - loss: 0.4001 - accuracy: 0.8597 - val_loss: 0.3206 - val_accuracy: 0.8865
Epoch 2/5
469/469 [==============================] - 122s 260ms/step - loss: 0.2699 - accuracy: 0.9026 - val_loss: 0.2868 - val_accuracy: 0.8965
Epoch 3/5
410/469 [=========================>....] - ETA: 14s - loss: 0.2290 - accuracy: 0.9167

**CNN TensorBoard Logs**

In [ ]:
%tensorboard --logdir logs/cnn

**CNN Best Model Details**

In [ ]:
best_cnn_hps = bayesian_cnn_tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_cnn_hps.values)

In [ ]:
simple_cnn_best_model = bayesian_cnn_tuner.get_best_models(num_models=1)[0]
simple_cnn_eval = simple_cnn_best_model.evaluate(x_test, y_test)
print(simple_cnn_eval[1])

**FNN**

In [ ]:
bayesian_fnn_tuner = BayesianOptimization(
    build_feedforward_nn,
    objective='val_accuracy',
    max_trials=5,
    directory='bayesian_optimization_fnn_logs',
    project_name='bayesian_optimization'
)

In [ ]:
bayesian_fnn_tuner.search(x=x_train, y=y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test),
                        callbacks=[tensorboard_fnn_callback])

**FNN TensorBoard Logs**

In [ ]:
%tensorboard --logdir logs/fnn

**FNN Best Model Details**

In [ ]:
best_fnn_hps = bayesian_fnn_tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_fnn_hps.values)

In [ ]:
feedforward_nn_best_model = bayesian_fnn_tuner.get_best_models(num_models=1)[0]
feedforward_nn_eval = feedforward_nn_best_model.evaluate(x_test, y_test)
print(feedforward_nn_eval[1])

**Model Comparison for Optimal Model**

In [ ]:
# Determine and save the better model
if simple_cnn_eval[1] > feedforward_nn_eval[1]:  # comparing validation accuracy (index 1)
    better_model = simple_cnn_best_model
    model_name = "simple_cnn"
else:
    better_model = feedforward_nn_best_model
    model_name = "feedforward_nn"


**Save Best Model**

In [ ]:
# Save the better model
better_model.save(f'best_{model_name}_fashion_mnist_model')